# Compare SentimentAnalysis models

### 1. Dependencies & Pre-processing
To derive files test_X, test_Y, I split the labels and review for each line in the original Amazon Reviews dataset. I then replace "\__label\__" with null, and only keep the numerical sentiment label. I write the label and review data into 2 separate files.

In [1]:
using TextAnalysis

testset = Array{String, 1}(undef, 400000)
file1 = open("test_X.txt")
testlabels = Array{Int8, 1}(undef, 400000)
file2 = open("test_Y.txt")

global cnt = 1
for i in readlines(file1), j in readlines(file2)
    testlabels[cnt] = parse(Int8, j)
    if length(i) > 500
        try
            testset[cnt] = i[1:500]
        catch e
            # char at index 500 is of iso-8859-1 not Unicode!
            testset[cnt] = i[1:499]
        end
    else
        testset[cnt] = i
    end
    global cnt += 1
end

close(file1)
close(file2)

### 2. Load pretrained TextAnalysis model & test
After testing, I find the model accuracy to be 50.0, without training on the Amazon Review dataset (only pretrained on the IMDB dataset)

In [ ]:
model = SentimentAnalyzer()
global total_correct = 0
total = 400000
global cnt = 1

for i in 1:400000
    input = Document(testset[i])
    prepare!(input, strip_stopwords)
    pred = Int8.(round(model(input)))
    if (pred+1) == testlabels[i]
        global total_correct += 1
    end
    if i % 1000 == 0 
        println("[", cnt, "]: ", total_correct/i*100)
        global cnt += 1
    end
end
println("The accuracy of TextAnalysis' pretrained model is ", total_correct/total*100)


[1]: 50.2
[2]: 52.300000000000004
[3]: 52.1
[4]: 51.225
[5]: 51.300000000000004
[6]: 51.6
[7]: 51.31428571428571
[8]: 51.0
[9]: 51.233333333333334
[10]: 51.24999999999999
[11]: 51.018181818181816
[12]: 50.9
[13]: 50.63846153846154
[14]: 50.7
[15]: 50.71333333333333
[16]: 50.8625
[17]: 50.87647058823529
[18]: 51.022222222222226
[19]: 51.0421052631579
[20]: 51.07000000000001
[21]: 51.26666666666667
[22]: 51.17272727272727
[23]: 51.221739130434784
[24]: 51.324999999999996
[25]: 51.37199999999999
[26]: 51.330769230769235
[27]: 51.32962962962962
[28]: 51.29642857142858
[29]: 51.110344827586204
[30]: 51.11
[31]: 51.07419354838709
[32]: 51.009375
[33]: 50.92121212121212
[34]: 50.964705882352945
[35]: 50.96857142857143
[36]: 50.93611111111112
[37]: 50.891891891891895
[38]: 50.931578947368415
[39]: 50.9051282051282
[40]: 50.897499999999994
[41]: 50.88292682926829
[42]: 50.878571428571426
[43]: 50.818604651162794
[44]: 50.79772727272728
[45]: 50.760000000000005
[46]: 50.73043478260869
[47]: 50.7